In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib.pyplot import figure
sns.set()

In [2]:
data = pd.read_csv("data/train.csv")
data.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD         Normal     208500  
1   2007        WD         Normal     181500  
2   2008        WD         Normal     223500  
3   2006        WD        Abnorml     140000  
4   2008        WD         Normal     250000  

[5 rows x 81 columns]

In [3]:
from sklearn.model_selection import train_test_split
X = data.drop('SalePrice', axis=1)
y = data.SalePrice
X_test = pd.read_csv('data/test.csv')
X_train, X_val ,y_train, y_val = train_test_split(X,y, random_state=16)
full_X_train = pd.concat([X_train,y_train],axis=1)

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer

class PandasTransform(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.columns_dropped = ['LotFrontage', 'Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature','YearRemodAdd','YearBuilt','MoSold', 'YrSold','GarageType','GarageYrBlt','GarageFinish','GarageArea','GarageCond','Exterior2nd']
        self.cat_col = ['MSZoning','Street','LotShape',
         'LandContour','Utilities','LotConfig','LandSlope','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
         'Heating','HeatingQC','CentralAir','Electrical','KitchenQual','Functional','GarageQual','PavedDrive','SaleType','SaleCondition']
        self.qc_order = {"na":0,"po":1, "fa":2, "ta":3,'gd':4,'ex':5}
        self.col_to_ordinal = ['ExterQual','ExterCond','BsmtQual','BsmtCond','HeatingQC','KitchenQual','GarageQual']
        self.catcol_wout_ordinal = ['MSZoning','Street', 'LotShape',
     'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'MasVnrType', 'Foundation', 'BsmtExposure', 'BsmtFinType1',
     'BsmtFinType2', 'Heating', 'CentralAir', 'Electrical', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']
        self.g1 = [['MeadowV', 'BrDale', 'IDOTRR'],
         ['OldTown',
          'BrkSide',
          'Edwards',
          'Sawyer',
          'NPkVill',
          'SWISU',
          'NAmes',
          'Blueste',
          'Mitchel',
          'SawyerW'],
         ['NWAmes',
          'Gilbert',
          'Blmngtn',
          'CollgCr',
          'Crawfor',
          'ClearCr',
          'Somerst',
          'Veenker'],
         ['Timber', 'StoneBr'],
         ['NridgHt', 'NoRidge']]
        self.g2 = ['BrkComm',
         ['AsphShn', 'CBlock', 'AsbShng'],
         ['Wd Sdng', 'MetalSd', 'WdShing', 'HdBoard'],
         ['Plywood', 'Stucco', 'BrkFace'],
         ['VinylSd', 'CemntBd'],
         ['ImStucc', 'Stone']]
    def fit(self, X, y= None):
        return self
    def transform(self, X):
        data = X.copy()
        data = data.drop(self.columns_dropped,axis=1)
        for col in self.cat_col:
            data[col] = data[col].str.lower()
        data[self.col_to_ordinal] = data[self.col_to_ordinal].fillna("na") 
        data[self.col_to_ordinal] = data[self.col_to_ordinal].replace(self.qc_order)
        
        for x in range(len(self.g1)):
            data = data.replace(self.g1[x], x)
        for x in range(len(self.g2)):
            data = data.replace(self.g2[x], x)
        data = data.set_index(data['Id'])
        data = data.drop('Id',axis=1)
        return data 

In [5]:
a = PandasTransform()
X_train = a.transform(X_train)

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

num_col = ['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', 'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageQual', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']
cat_col = ['MSZoning','Street', 'LotShape','LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'MasVnrType', 'Foundation', 'BsmtExposure', 'BsmtFinType1','BsmtFinType2', 'Heating', 'CentralAir', 'Electrical', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']
cat_pipeline = Pipeline([

    ('cat_encoder', OneHotEncoder(handle_unknown='ignore'))
    ])
preprocessor = ColumnTransformer(transformers=[
    ('cat', cat_pipeline, cat_col),
    ('num', SimpleImputer(), num_col)
    ])

In [7]:
def make_pipeline(model):
    return Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
        ])

In [8]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
pipeline_xgb = make_pipeline( XGBRegressor() )


param_grid = {'model__n_estimators':(100,500),
              'model__learning_rate':(0.05,0.1,0.5,1)
              }


# cv_model_search = GridSearchCV(pipeline_xgb, 
#                                param_grid, 
#                                cv = 3,
#                                refit=False, 
#                                verbose = 1,
#                               error_score='raise') 

# # Fit all (1) models defined in our model-search object
# best = cv_model_search.fit(X_train,y_train)

C:\Users\zubik\anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [13]:
best.cv_results_['mean_test_score']

array([0.86951303, 0.87386984, 0.87989079, 0.87876267, 0.84899917,
       0.84898814, 0.7197716 , 0.71977177])

In [17]:
best.cv_results_['params']

[{'model__learning_rate': 0.05, 'model__n_estimators': 100},
 {'model__learning_rate': 0.05, 'model__n_estimators': 500},
 {'model__learning_rate': 0.1, 'model__n_estimators': 100},
 {'model__learning_rate': 0.1, 'model__n_estimators': 500},
 {'model__learning_rate': 0.5, 'model__n_estimators': 100},
 {'model__learning_rate': 0.5, 'model__n_estimators': 500},
 {'model__learning_rate': 1, 'model__n_estimators': 100},
 {'model__learning_rate': 1, 'model__n_estimators': 500}]

In [10]:
param_grid2 = {'model__n_estimators':(100,500,1000),
              'model__learning_rate':(0.05,0.075,0.08,0.09)
              }


cv_model_search2 = GridSearchCV(pipeline_xgb, 
                               param_grid2, 
                               cv = 3,
                               refit=True, 
                               verbose = 1,
                              error_score='raise') 

# Fit all (1) models defined in our model-search object
cv_model_search2 = cv_model_search2.fit(X_train,y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


In [11]:
cv_model_search2.cv_results_['mean_test_score']

array([0.86951303, 0.87386984, 0.87366273, 0.87989778, 0.87964166,
       0.87960392, 0.87657063, 0.87711472, 0.87703585, 0.87573612,
       0.87474257, 0.87470041])

In [12]:
cv_model_search2.cv_results_['params']

[{'model__learning_rate': 0.05, 'model__n_estimators': 100},
 {'model__learning_rate': 0.05, 'model__n_estimators': 500},
 {'model__learning_rate': 0.05, 'model__n_estimators': 1000},
 {'model__learning_rate': 0.075, 'model__n_estimators': 100},
 {'model__learning_rate': 0.075, 'model__n_estimators': 500},
 {'model__learning_rate': 0.075, 'model__n_estimators': 1000},
 {'model__learning_rate': 0.08, 'model__n_estimators': 100},
 {'model__learning_rate': 0.08, 'model__n_estimators': 500},
 {'model__learning_rate': 0.08, 'model__n_estimators': 1000},
 {'model__learning_rate': 0.09, 'model__n_estimators': 100},
 {'model__learning_rate': 0.09, 'model__n_estimators': 500},
 {'model__learning_rate': 0.09, 'model__n_estimators': 1000}]

In [16]:
X_test = a.transform(X_test)
pred = cv_model_search2.predict(X_test)

output = pd.DataFrame({'Id': X_test.index],
                       'SalePrice': pred})
output.to_csv('./submission.csv', index=False)

KeyError: "['LotFrontage', 'Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature', 'YearRemodAdd', 'YearBuilt', 'MoSold', 'YrSold', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageArea', 'GarageCond', 'Exterior2nd'] not found in axis"

In [19]:
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': pred})
output.to_csv('./submission.csv', index=False)